# EXTERNAL

In [1]:
#@title OpenBioLink Module Install EXTERNAL{ form-width: "15%" }
! pip install openbiolink


     |████████████████████████████████| 226 kB 5.4 MB/s 


In [2]:
#@title PyG 113 Installation EXTERNAL{ form-width: "15%" }
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/rusty1s/pytorch_geometric.git

     |████████████████████████████████| 7.9 MB 5.0 MB/s 
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 407 kB 5.3 MB/s 
     |████████████████████████████████| 45 kB 2.3 MB/s 


# HEADER

In [3]:
#@title Module Imports HEADER { form-width: "15%" }
from openbiolink.obl2021 import OBL2021Dataset, OBL2021Evaluator
import torch
from torch.nn import Module,\
                     ModuleList,\
                     Embedding,\
                     BatchNorm1d,\
                     LogSoftmax,\
                     Softmax,\
                     Linear,\
                     NLLLoss,\
                     CrossEntropyLoss
from torch.optim import Adam
import torch.nn.functional as F
import torch_geometric as PyG
from torch_geometric.data import Data
from torch_geometric.nn.conv import RGCNConv
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from typing import NoReturn
import typing
from enum import Enum
from collections import defaultdict

# GLOBAL 
<!-- ###now CPU -->

In [16]:
#@title Global Variables CLASS now CPU{ form-width: "15%" }
class Global(Enum):
  HEAD_INDEX = 0
  RELATION_INDEX = 1
  TAIL_INDEX = 2
  FEATURE_ENG = 'one-hot'
  NUM_RELATIONS = 28
  DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
  # DEVICE = 'cpu'
  MINI_BATCH_SIZE = 32
  EVAL_BATCH_SIZE = 100
  VAL_HITS_AT = 50
  TEST_HITS_AT = 50


dataset = OBL2021Dataset()
kg = torch.cat((dataset.training, dataset.validation, dataset.testing), dim=0)
sorted_on_tails_indcs_kg = torch.sort(kg[:, Global.TAIL_INDEX.value])[1]
kg_sorted_tails = kg[sorted_on_tails_indcs_kg]
sorted_on_heads_indcs_kg = torch.sort(kg[:, Global.HEAD_INDEX.value])[1]
kg_sorted_heads = kg[sorted_on_heads_indcs_kg]
train_set = dataset.training.to(Global.DEVICE.value) # torch.tensor of shape(num_train,3)
val_set = dataset.validation.to(Global.DEVICE.value) # torch.tensor of shape(num_val,3)
test_set = dataset.testing.to(Global.DEVICE.value)   # torch.tensor of shape(num_train,3)

Dataset found in /content/obl2021, omitting download...


In [5]:
dataset.get_test_batches(100)

(1810,
 <generator object split_list_in_batches_iter.<locals>.<genexpr> at 0x7fe3f4ab0f50>)

# GNN Model and Hyperparameters

In [6]:
#@title GNN Model CLASS{ form-width: "10%" }
class GNN(Module):
  def __init__(self, conv_dims: list, fully_connected_dims: list, x_feature:str, dropout: dict, embedding_dims: tuple=None)-> NoReturn:
    super(GNN, self).__init__()
    self.mode = None # 'train' or 'test' or 'dev' later 
    self.num_relations = Global.NUM_RELATIONS.value
    self.dropout = dropout
    self.x_feature = x_feature
    # relation_weights_list = [Linear(conv_dims[-1], conv_dims[-1], bias=False) for _ in range(28)]
    # self.relation_weights = ModuleList(relation_weights_list)
    if x_feature == 'one-hot':
      #one-hot to latent
      self.entity_embedder = Embedding(embedding_dims[0], embedding_dims[1])
      self.relation_embedder = Embedding(28, conv_dims[-1])
      first_conv_layer = [RGCNConv(embedding_dims[1], conv_dims[0], self.num_relations)]
    elif x_feature == 'identity':
      first_conv_layer = [RGCNConv(1, conv_dims[0], self.num_relations)]
    conv_list = first_conv_layer + \
                                [
                                  RGCNConv(conv_dims[i], conv_dims[i+1], self.num_relations)
                                  for i in range(len(conv_dims[:-1]))
                                ]
  

    # fully_connected_list =   [
    #                             Linear(conv_dims[-1], fully_connected_dims[0])
    #                          ] + \
    #                          [
    #                             Linear(fully_connected_dims[i], fully_connected_dims[i+1])
    #                             for i in range(len(fully_connected_dims[:-1]))
    #                          ] 

    #graph conv layers
    self.conv_layers = ModuleList(conv_list)

    #fully connected dense layers
    # self.fully_connected_layers = ModuleList(fully_connected_list)

    # self.classifier = LogSoftmax(dim=1)

    
  def reset_parameters(self):
    self.entity_embedder.reset_parameters()
    self.relation_embedder.reset_parameters()
    for conv in self.conv_layers:
        conv.reset_parameters()
    # for fc in self.fully_connected_layers:
    #     fc.reset_parameters()
      

  def forward(self, data: Data) -> torch.Tensor:
    edge_index = data.edge_index_messaging
    x = data.x
    edge_type = data.edge_type_messaging

    # print(x.shape)
    # return

    # latent = list()
      # last_index = 0
      # for i in range(0, x.shape[0] - 1024, 1024):
      #   latent.append(self.embed(x[i: i+1024, :]))
      #   last_index = i
      # last_latent = self.embed(x[last_index:, : ]).reshape(-1, self.embed.weight.shape[1])
      # partial_latent_space = torch.stack(latent).reshape(-1, self.embed.weight.shape[1])
      # x = torch.cat(
      #     (partial_latent_space, last_latent),
      #     dim=0 
      # )

    ####################################### ONE - HOT #######################################
    if self.x_feature == 'one-hot':
      x = self.entity_embedder(x).reshape(self.entity_embedder.weight.shape[0], -1)

      
    ####################################### Encoder: RGCN #######################################
      
    ############################################## IDENTITY ################################################
    elif self.x_feature == 'identity':
      x = torch.ones(x.shape[0], 1) ############
      if self.training:
        x = F.dropout(x, p=self.dropout["emb"])
      print(f'Embedding {x.shape}')
    ####################################### Encoder: RGCN #######################################
    for conv in self.conv_layers[:-1]:
      x = conv(x, edge_index=edge_index, edge_type=edge_type)
      x = F.relu(x)
      if self.training:
        x = F.dropout(x, p=self.dropout["conv"])
    x = self.conv_layers[-1](x, edge_index, edge_type)
    if self.training:
      x = F.dropout(x, p=self.dropout["conv"])

    # x[head] + v[relation] -> x[tail]
    # < x[head] . v[relation] . x[tail] >

    
    # return
    # supervision_edge_index = torch.cat((data.edge_index_supervision, data.edge_index_negative), dim=1)
    # supervision_edge_type = torch.cat((data.edge_type_supervision, data.edge_type_negative))
    # which_one_is_which = torch.cat(
    #     (torch.zeros_like(data.edge_type_supervision), torch.ones_like(data.edge_type_negative))
    # ).to(Global.DEVICE.value)

    # random_permutation = torch.randperm(supervision_edge_type.shape[0])
    # heads = supervision_edge_index[0][random_permutation]
    # tails = supervision_edge_index[1][random_permutation]
    # relations = supervision_edge_type[random_permutation]
    # which_one_is_which = which_one_is_which[random_permutation]

    # n_scores = list()
    # p_scores = list()
    # for index in range(heads.shape[0]):
    #   weights = self.relation_weights[relations[index]]
    #   head = x[heads[index]]
    #   tail = x[tails[index]] 
    #   score = weights(head) @ tail 
    #   if which_one_is_which[index].item() == 1:
    #     p_scores.append(score)
    #   elif which_one_is_which[index].item() == 0:
    #     n_scores.append(score)
    # return torch.stack(p_scores), torch.stack(n_scores)


    # for index, p_edge_relation in enumerate(data.edge_type_supervision):
    #   weights = self.relation_weights[p_edge_relation]
    #   head = x[data.edge_index_supervision[0, index]]
    #   tail = x[data.edge_index_supervision[1, index]]
    #   score = weights(head) @ tail 
    #   p_scores.append(score)
    
    ####################################### Decoder #######################################

    positive_heads = x[data.edge_index_supervision[0]]
    positive_tails = x[data.edge_index_supervision[1]]
    positive_relations = self.relation_embedder(data.edge_type_supervision)

    negative_heads = x[data.edge_index_negative[0]]
    negative_tails = x[data.edge_index_negative[1]]
    negative_relations = self.relation_embedder(data.edge_type_negative)

    return (positive_heads * positive_relations * positive_tails).sum(dim=1), \
           (negative_heads * negative_relations * negative_tails).sum(dim=1)
    

    


In [7]:
#@title Model and Hyperparameters MAIN { form-width: "15%" }
model = GNN(
    x_feature='one-hot',
    conv_dims=[32, 64, 64, 64],
    embedding_dims=(dataset.candidates.max() + 1, 32),
    fully_connected_dims=[1], 
    dropout={
        "emb": 0.1,
        "conv": 0.2,
        "fc": 0.01
    }
).to(Global.DEVICE.value)
print(model)
loss_fn = torch.nn.BCEWithLogitsLoss()
opt = Adam(model.parameters())

GNN(
  (entity_embedder): Embedding(180992, 32)
  (relation_embedder): Embedding(28, 64)
  (conv_layers): ModuleList(
    (0): RGCNConv(32, 32, num_relations=28)
    (1): RGCNConv(32, 64, num_relations=28)
    (2): RGCNConv(64, 64, num_relations=28)
    (3): RGCNConv(64, 64, num_relations=28)
  )
)


#FUNCTIONS

In [8]:

#@title #TODO  use psuedo_negative method mini_batch_maker() FUNCTION #TODO { form-width: "15%" }

def mini_batch_maker(messaging, supervision, candidates, x_feature='one-hot'):
  heads = supervision[:, 0]
  relations = supervision[:, 1]
  tails = supervision[:, -1]

  ct_size = supervision.shape[0] // 2
  ch_size = supervision.shape[0] - ct_size

  negative_samples_corrupted_tails = torch.vstack(
      (
          heads[: ct_size], 
          relations[: ct_size], 
          torch.multinomial(
              candidates.type(torch.float).to(Global.DEVICE.value), 
              ct_size
          )
      )
  ).t().contiguous()

  negative_samples_corrupted_heads = torch.vstack(
      ( 
          torch.multinomial(
              candidates.type(torch.float).to(Global.DEVICE.value), 
              ch_size
          ), 
          relations[ch_size:], 
          tails[ch_size:]
      )
  ).t().contiguous()

  negative_samples = torch.cat(
      (negative_samples_corrupted_heads, negative_samples_corrupted_tails),
      dim=0
  )

  # for __index, candidate in enumerate(negative_samples):
  #   is_corrupt = not any(np.equal(knowledge_graph.numpy(), candidate.numpy().tolist()).all(1))
  #   if not is_corrupt:
  #     print(f'{candidate} is not a true corrupted edge')

  graph = graph_data_maker(
      messaging=messaging,
      supervision=supervision,
      negative_samples=negative_samples,
      x=candidates.to(Global.DEVICE.value),
      x_feature='one-hot'
  )

  return graph

In [9]:
#@title graph_data_maker(dataset, x_feature) FUNCTION{ form-width: "15%" }
def graph_data_maker(messaging: torch.Tensor, supervision: torch.Tensor, negative_samples: torch.Tensor, x:torch.Tensor, x_feature: str=Global.FEATURE_ENG.value, check_for_correctness: bool=False) -> Data:
  relation_idx = Global.RELATION_INDEX.value
  head_idx = Global.HEAD_INDEX.value
  tail_idx = Global.TAIL_INDEX.value
  graph_data = Data(
        x=x.reshape(-1, 1),
        edge_index_messaging=messaging[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_messaging=messaging[:, relation_idx],
        edge_index_supervision=supervision[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_supervision=supervision[:, relation_idx],
        edge_index_negative=negative_samples[:, (head_idx, tail_idx)].t().contiguous(),
        edge_type_negative=negative_samples[:, (relation_idx)]
    )
  return graph_data


In [10]:
#@title find_start_and_end_indcs() PRIVATE-FUNCTION { form-width: "15%" }
def find_start_and_end_indcs(entity, entity_type, kg_sorted):
  up = kg_sorted.shape[0]
  down = 0
  indx = kg_sorted.shape[0] // 2
  found = False
  if entity_type == Global.TAIL_INDEX.value:
    bound = 1
  elif entity_type == Global.HEAD_INDEX.value:
    bound = 0
  while down + bound < up:
    if kg_sorted[indx][entity_type].item() == entity:
      found = True
      break 
    elif kg_sorted[indx][entity_type].item() >= entity: 
      up = indx
      indx = (up + down) // 2
    else:
      down = indx 
      indx = (up + down) // 2
  if not found:
    return None
  while 1:
    indx += 1
    try:
      if not kg_sorted[indx][entity_type].item() == entity:
        indx -= 1
        end_indx = indx
        break
    except:
      end_indx = indx - 1
      break
  

  while 1:
    indx -= 1
    try:
      if not kg_sorted[indx][entity_type].item() == entity:
        indx += 1
        start_indx = indx
        break
    except:
      start_indx = indx + 1
      break
  return torch.tensor(range(start_indx, end_indx + 1))

In [11]:
#@title get_psuedo_negative_entities() FUNCTIONS { form-width: "15%" }
def get_psuedo_negative_entities(entity, corrupt_at, kg_sorted):
  if corrupt_at == Global.HEAD_INDEX.value:
    entity_type = Global.TAIL_INDEX.value 
  elif corrupt_at == Global.TAIL_INDEX.value:
    entity_type = Global.HEAD_INDEX.value
  indcs = find_start_and_end_indcs(entity, entity_type, kg_sorted=kg_sorted)
  if indcs is not None:
    fact_triplets_entities = kg_sorted[indcs][:, corrupt_at]
    return torch.tensor([ _ for _ in range(dataset.candidates.shape[0]) if _ not in fact_triplets_entities.tolist()]).to(Global.DEVICE.value)
  else:
    return dataset.candidates.to(Global.DEVICE.value)


In [12]:
#@title train(model, graph, optimizer, loss_fn) FUNCTION{ form-width: "15%" }
def train(model: GNN, graph: Data, optimizer: torch.optim, loss_fn:torch.nn.modules.loss) -> tuple((torch.float, torch.Tensor, torch.Tensor)):
  optimizer.zero_grad()
  model.train()
  positive, negative = model(graph)
  loss = loss_fn(
            negative, 
            torch.zeros(graph.edge_index_negative.shape[1]).to(Global.DEVICE.value)
         ) + loss_fn(
                positive,
                torch.ones(graph.edge_index_supervision.shape[1]).to(Global.DEVICE.value)
            )
  loss.backward()
  optimizer.step()
  return loss.item(), torch.sigmoid(positive), torch.sigmoid(negative)

In [17]:
#@title evaluation_rank() PRIVATE-FUNCTION { form-width: "15%" }
@torch.no_grad()
def evaluation_rank(model, eval_set, messaging_set):
  h_pred_top10 = list()
  t_pred_top10 = list()

  for eval_triplet in tqdm(eval_set):
    head = eval_triplet[Global.HEAD_INDEX.value].item()
    relation = eval_triplet[Global.RELATION_INDEX.value].item()
    tail = eval_triplet[Global.TAIL_INDEX.value].item()
    corrupted_tails = get_psuedo_negative_entities(
        entity=head, 
        corrupt_at=Global.TAIL_INDEX.value, 
        kg_sorted=kg_sorted_heads
    )
    num_psuedo_negative_triples = corrupted_tails.shape[0]
    psuedo_negative_triplets_tail = torch.vstack(
        (
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * head,
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * relation,
            corrupted_tails
        )
    ).t().to(Global.DEVICE.value)

    corrupted_heads = get_psuedo_negative_entities(
        entity=tail, 
        corrupt_at=Global.HEAD_INDEX.value, 
        kg_sorted=kg_sorted_tails
    )
    num_psuedo_negative_triples = corrupted_heads.shape[0]
    psuedo_negative_triplets_head = torch.vstack(
        (
            corrupted_heads,
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * relation,
            torch.ones(num_psuedo_negative_triples).to(Global.DEVICE.value).type(torch.long) * tail
        )
    ).t().to(Global.DEVICE.value)

    # eval_triplet: (h, r, t)
    # psuedo_negative_triplets_head: (h′, r, t) for all h′
    # psuedo_negative_triplets_tail: (h, r, t′) for all t′

    # train_set being the messaging graph, calculate the score for (h, r, t)
    # train_set being the messaging graph, calculate the scores for all (h′, r, t)
    # train_set being the messaging graph, calculate the scores for all (h, r, t′)

    graph_data_for_object_tail = graph_data_maker(
      messaging=messaging_set,
      supervision=eval_triplet.reshape(1, 3),
      negative_samples=psuedo_negative_triplets_tail,
      x=dataset.candidates.to(Global.DEVICE.value),
      x_feature='one-hot'
    )
    graph_data_for_object_head = graph_data_maker(
            messaging=messaging_set,
            supervision=eval_triplet.reshape(1, 3),
            negative_samples=psuedo_negative_triplets_head,
            x=dataset.candidates.to(Global.DEVICE.value),
            x_feature='one-hot'
    )
    model.eval()
    scores_for_object_tail = torch.cat(model(graph_data_for_object_tail))
    model.eval()
    scores_for_object_head = torch.cat(model(graph_data_for_object_head))

    sorted_by_scores_tails_indcs = torch.sort(scores_for_object_tail, descending=True)[1]
    tail_objects = torch.cat((graph_data_for_object_tail.edge_index_supervision[1], graph_data_for_object_tail.edge_index_negative[1]))
    top10_tails = tail_objects[sorted_by_scores_tails_indcs][:10]
    t_pred_top10.append(top10_tails)

    sorted_by_scores_heads_indcs = torch.sort(scores_for_object_head, descending=True)[1]
    head_objects = torch.cat((graph_data_for_object_head.edge_index_supervision[0], graph_data_for_object_head.edge_index_negative[0]))
    top10_heads = head_objects[sorted_by_scores_heads_indcs][:10]
    h_pred_top10.append(top10_heads)

  model.train()
  return torch.stack(h_pred_top10), torch.stack(t_pred_top10)

In [14]:
#@title evaluate_hits_at_10(model, mode) FUNCTION{ form-width: "15%" }
def evaluate_hits_at_10(model: GNN, mode:str) -> torch.float:
  if mode == 'validation':
    head_objects_top10, tail_objects_top10 = evaluation_rank(model, val_set, train_set)
    return OBL2021Evaluator().eval(
            head_objects_top10,
            tail_objects_top10,
            val_set,
            False
          )
  elif mode == 'testing':
    head_objects_top10, tail_objects_top10 = evaluation_rank(model, test_set, torch.cat((train_set, val_set), dim=0))
    return OBL2021Evaluator().eval(
            head_objects_top10,
            tail_objects_top10,
            test_set,
            False
          )


#Trainer

In [ ]:
# @title Parameter and Hyperparameter Tuning MAIN{ form-width: "10%" }
model.reset_parameters()
try:
  for epoch in range(5):
    iteration = 0
  ############################## TRAIN Graph Maker #####################################
    for i in tqdm(range(0, train_set.shape[0] - Global.MINI_BATCH_SIZE.value, Global.MINI_BATCH_SIZE.value)):
      iteration += 1
      train_supervision = train_set[i: i + Global.MINI_BATCH_SIZE.value, :]
      train_messaging = torch.cat(
          (train_set[: i, :], train_set[i + Global.MINI_BATCH_SIZE.value: , :]),
          dim=0
      )
      train_graph = mini_batch_maker(
          train_messaging,
          train_supervision,
          dataset.candidates,
          'one-hot'
      )
      loss, positive_score, negative_score = train(model, train_graph, opt, loss_fn)
      if iteration % 50 == 0:
        print(' ')
        agg_p_score = positive_score.sum().item()
        agg_n_score = negative_score.sum().item()
        print(f'Train Batch {iteration}:')
        print(f'Loss:        {loss: .4f}')
        print(f'Agg P-Score: {agg_p_score: .4f}')
        print(f'Agg N-Score: {agg_n_score: .4f}')
        print(f'===' * 25)
      if iteration % 5000 == 0:
        pass
except KeyboardInterrupt:
  pass


  0%|          | 50/131000 [01:07<49:10:43,  1.35s/it]

 
Train Batch 50:
Loss:         2.3223
Agg P-Score:  21.1231
Agg N-Score:  12.1451


  0%|          | 100/131000 [02:15<49:14:10,  1.35s/it]

 
Train Batch 100:
Loss:         1.5020
Agg P-Score:  25.4672
Agg N-Score:  11.9946


  0%|          | 150/131000 [03:23<49:06:27,  1.35s/it]

...

  0%|          | 250/131000 [05:38<49:03:45,  1.35s/it]

 
Train Batch 250:
Loss:         1.1897
Agg P-Score:  24.1424
Agg N-Score:  12.8308


  0%|          | 300/131000 [06:45<49:04:23,  1.35s/it]

 
Train Batch 300:
Loss:         1.0038
Agg P-Score:  25.0980
Agg N-Score:  11.9898


  0%|          | 307/131000 [06:55<49:08:24,  1.35s/it]

# TEST EVALUATION

In [18]:
try:
  evaluate_hits_at_10(model, 'testing')
except KeyboardInterrupt:
  pass

  0%|          | 0/180964 [00:18<?, ?it/s]


# ADD-ONS


In [ ]:
#@title add_tail_to_head(data_split) FUNCTION { form-width: "15%" }
def add_tail_to_head(data_split: torch.Tensor, plus: int=28) -> torch.Tensor:
  heads = data_split[:, (0)]
  tails = data_split[:, (-1)]
  relations = data_split[:, (1)]
  
  tail_to_head = torch.vstack(
      (tails, relations + plus, heads)
  ).t()

  return torch.cat(
      (data_split, tail_to_head),
      dim=0
  )



In [ ]:
#@title checksum(data_subset, graph_data) FUNCTION { form-width: "15%" }
def checksum(data_subset: torch.Tensor, graph_data: Data, step: int=500) -> NoReturn:
  for i in range(step, data_subset.shape[0], step):
    graph_head_indcs = graph_data.edge_index[0, i-step:i]
    data_head_entities = data_subset[i-step:i, 0]
    graph_head_entities = graph_data.x[graph_head_indcs].reshape(-1)

    checksum_head = (graph_head_entities == data_head_entities).sum().item()
    if not checksum_head == step:
      print('head')
      print(i)
      break

    graph_relation_indcs = graph_data.edge_type[i-step:i]
    data_relation_types = data_subset[i-step:i, 1]

    checksum_relation = (graph_relation_indcs == data_relation_types).sum().item()
    if not checksum_relation == step:
      print('relation')
      print(i)
      break

    graph_tail_indcs = graph_data.edge_index[1, i-step:i]
    data_tail_entities = data_subset[i-step:i, 2]
    graph_tail_entities = graph_data.x[graph_tail_indcs].reshape(-1)

    checksum_tail = (graph_tail_entities == data_tail_entities).sum().item()
    if not checksum_head == step:
      print('tail')
      print(i)
      break

  else:
    print('All clear :)')

In [ ]:
#@title partial_graph_data_maker(dataset, x_feature) FUNCTION{ form-width: "15%" }
def partial_graph_data_maker(messaging: torch.Tensor, supervision: torch.Tensor, negative_samples: torch.Tensor, x_feature: str=Global.FEATURE_ENG.value, check_for_correctness: bool=False) -> Data:
  relation_idx = Global.RELATION_INDEX.value
  head_idx = Global.HEAD_INDEX.value
  tail_idx = Global.TAIL_INDEX.value


  if x_feature == 'one-hot':
    one_hot_index = 0
    edge_index_list_flat = []
    seen_dict = defaultdict(lambda: -1)
    edge_index_flat = messaging[:, (head_idx, tail_idx)].reshape(-1)

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_messaging = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    edge_index_flat = supervision[:, (head_idx, tail_idx)].reshape(-1)
    edge_index_list_flat = []

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_supervision = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    edge_index_flat = negative_samples[:, (head_idx, tail_idx)].reshape(-1)
    edge_index_list_flat = []

    for i in range(edge_index_flat.shape[0]):
      entity = edge_index_flat[i].item()
      if seen_dict[entity] == -1:
        edge_index_list_flat.append(one_hot_index)
        seen_dict[entity] = one_hot_index
        one_hot_index += 1
      else:
        edge_index_list_flat.append(seen_dict[entity])
      if i + 1 % 50000 == 0:
        print(f'Entity {i} encoded. {i / edge_index_flat.shape[0] * 100:.2f} %')
    edge_index_negative = torch.tensor(edge_index_list_flat).reshape(-1, 2)
    ##############################################################
    x = torch.tensor(list(seen_dict.keys())).reshape(-1, 1)

    graph_data = Data(
        x=x,
        edge_index_messaging=edge_index_messaging.t().contiguous(),
        edge_type_messaging=messaging[:, relation_idx],
        edge_index_supervision=edge_index_supervision.t().contiguous(),
        edge_type_supervision=supervision[:, relation_idx],
        edge_index_negative=edge_index_negative.t().contiguous(),
        edge_type_negative=negative_samples[:, (relation_idx)]
    )
  else:
    raise NotImplementedError('This functionality has not been implemented yet.')

  if check_for_correctness:
    checksum(dataset, graph_data)
  return graph_data

In [ ]:
#@title visualize_graph(graph_data, height, width) FUNCTION{ form-width: "15%" }
def visualize_graph(graph_data: Data, height: int=10, width:int=10) -> NoReturn:
  nx_graph = to_networkx(graph_data)

  pos = nx.spring_layout(nx_graph)

  fig = plt.gcf()
  fig.set_size_inches(width, height)

  edge_labels = dict()
  ei_np = graph_data.edge_index.t().numpy()
  for edge in nx_graph.edges():
    e = np.array(edge)
    idx = np.where(ei_np == e)[0][0]
    label = graph_data.edge_type[idx].item()
    edge_labels.update({edge: label})
    
  nx.draw_networkx_nodes(nx_graph, pos)
  nx.draw_networkx_edges(nx_graph, pos, connectionstyle='arc3,rad=0.2')
  nx.draw_networkx_labels(nx_graph, pos, labels={n:graph_data.x[n].item() for n in nx_graph})
  nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels)

  fig.show()


In [ ]:
#@title Visualization OPTIONAL-MAIN { form-width: "15%" }
data_subset = train_set[:2500, :]
graph_data = graph_data_maker(data_subset, 'one-hot')
visualize_graph(graph_data, 100, 100)